In [4]:
### This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/home/cristiandugacicu/projects/personal/zoomcamp'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

df_raw = pd.read_csv("/home/cristiandugacicu/projects/personal/zoomcamp/AB_NYC_2019.csv")

columns = ['neighbourhood_group','room_type','latitude','longitude','price','minimum_nights',
            'number_of_reviews','reviews_per_month','calculated_host_listings_count','availability_365']

df = df_raw[columns].fillna(0)


/home/cristiandugacicu/projects/personal/zoomcamp/AB_NYC_2019.csv.zip
/home/cristiandugacicu/projects/personal/zoomcamp/AB_NYC_2019.csv
/home/cristiandugacicu/projects/personal/zoomcamp/homework3.ipynb
/home/cristiandugacicu/projects/personal/zoomcamp/data.csv
/home/cristiandugacicu/projects/personal/zoomcamp/WA_Fn-UseC_-Telco-Customer-Churn.csv


In [6]:
# Q1
df.neighbourhood_group.value_counts()

Manhattan        21661
Brooklyn         20104
Queens            5666
Bronx             1091
Staten Island      373
Name: neighbourhood_group, dtype: int64

In [7]:
# Split the data

from sklearn.model_selection import train_test_split

df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)
print(len(df_train), len(df_val), len(df_test))

df_train = df_train.reset_index(drop="true")
df_val = df_val.reset_index(drop="true")
df_test = df_test.reset_index(drop="true")

y_full_train = df_full_train.price.values
y_train = df_train.price.values
y_val = df_val.price.values
y_test = df_test.price.values

y_train_log = np.log1p(df_train.price.values)
y_val_log = np.log1p(df_val.price.values)
y_test_log = np.log1p(df_test.price.values)
print("Total reccords after train/val/test random repartition:",len(y_train)+ len(y_val)+ len(y_test))


del df_test["price"]
del df_val["price"]
del df_train["price"]
df_train.dtypes

29337 9779 9779
Total reccords after train/val/test random repartition: 48895


neighbourhood_group                object
room_type                          object
latitude                          float64
longitude                         float64
minimum_nights                      int64
number_of_reviews                   int64
reviews_per_month                 float64
calculated_host_listings_count      int64
availability_365                    int64
dtype: object

In [8]:
#Q2 Corelation matrix

numerical = ["latitude", "longitude", "minimum_nights", "number_of_reviews", "reviews_per_month", "calculated_host_listings_count", "availability_365"]
categorical = ["neighbourhood_group", "room_type"]

from sklearn import metrics

# how much do we learn about "churn" by observing the "contract" variable:
df_train[numerical].corr("pearson")


,latitude,longitude,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
latitude,1.000000,0.080301,0.027441,-0.006246,-0.007159,0.019375,-0.005891
longitude,0.080301,1.000000,-0.060660,0.055084,0.134642,-0.117041,0.083666
minimum_nights,0.027441,-0.060660,1.000000,-0.076020,-0.120703,0.118647,0.138901
number_of_reviews,-0.006246,0.055084,-0.076020,1.000000,0.590374,-0.073167,0.174477
reviews_per_month,-0.007159,0.134642,-0.120703,0.590374,1.000000,-0.048767,0.165376
calculated_host_listings_count,0.019375,-0.117041,0.118647,-0.073167,-0.048767,1.000000,0.225913
availability_365,-0.005891,0.083666,0.138901,0.174477,0.165376,0.225913,1.000000


In [9]:
#Q3 

df_train["above_average"] = (y_train>=152).astype('int')
y_train_binary = df_train.above_average.values
y_val_binary = (y_val>=152).astype("int")

from sklearn import metrics

# how much do we learn about "churn" by observing the "contract" variable:
neighbourhood_group_mutual_score = metrics.mutual_info_score(df_train.above_average, df_train.neighbourhood_group)
room_type_mutual_score = metrics.mutual_info_score(df_train.above_average, df_train.room_type)
print("neighbourhood_group_mutual_score:",round(neighbourhood_group_mutual_score,2),"; room_type_mutual_score:",round(room_type_mutual_score,2))


neighbourhood_group_mutual_score: 0.05 ; room_type_mutual_score: 0.14


In [10]:
# Q4

from sklearn.feature_extraction import DictVectorizer
from sklearn import linear_model


def train_model_with(dv, dataFrame, y):
    dicts = dataFrame.to_dict(orient="records")
    X = dv.fit_transform(dicts)
    display("Training model for features:",dv.feature_names_)

    model = linear_model.LogisticRegression(solver='lbfgs', C=1.0, random_state=42)
    model.fit(X, y)
    return model

dv = DictVectorizer(sparse=False)

train_model = train_model_with(dv, df_train[categorical], y_train_binary)

#accuracy on val dataset:
dicts_val = df_val[categorical].to_dict(orient="records")
dv_val = DictVectorizer(sparse=False)
dv_val.fit(dicts_val)
X_val = dv_val.transform(dicts_val)
y_val_probability = train_model.predict_proba(X_val)[:,1]
# churn_decision = (y_pred >= 0.5)
y_val_probability
# print("Model accuracy based on test data:", (y_pred == y_test).mean())

above_average_prce_probability = (y_val_probability>=0.5).astype("int")

accuracy = (above_average_prce_probability == y_val_binary).mean()
print()
print("Model accuracy based on val data:", round(accuracy,3))



'Training model for features:'

['neighbourhood_group=Bronx',
 'neighbourhood_group=Brooklyn',
 'neighbourhood_group=Manhattan',
 'neighbourhood_group=Queens',
 'neighbourhood_group=Staten Island',
 'room_type=Entire home/apt',
 'room_type=Private room',
 'room_type=Shared room']


Model accuracy based on val data: 0.781


In [11]:
email = "35a89bd1cf9dc20f4a30d70cadf18c60942dfaff"
import sklearn
sklearn.__version__

'0.24.1'

In [16]:
# Q5

from sklearn.feature_extraction import DictVectorizer
from sklearn import linear_model


def train_model_with(dv, dataFrame, y):
    dicts = dataFrame.to_dict(orient="records")
    X = dv.fit_transform(dicts)
#     display("Training model for features:",dv.feature_names_)

    model = linear_model.LogisticRegression(solver='lbfgs', C=1.0, random_state=42, max_iter=10000)
    model.fit(X, y)
    return model

def get_accuracy(dv1, model, df, y):
    dicts_val1 = df.to_dict(orient="records")
    X_val1 = dv1.fit_transform(dicts_val1)
    y_val_probability1 = model.predict_proba(X_val1)[:,1]

    above_average_prce_probability1 = (y_val_probability1>=0.5).astype("int")

    accuracy1 = (above_average_prce_probability1 == y).mean()
    return accuracy1

dv5 = DictVectorizer(sparse=False)
train_model_all_features = train_model_with(dv5, df_train[categorical+numerical], y_train_binary)
accuracy0 = get_accuracy(dv5, train_model_all_features, df_val[categorical+numerical], y_val_binary)
print("Model accuracy based on val data without:", round(accuracy0,2))

for c in (categorical + numerical):
    dvC = DictVectorizer(sparse=False)
    new_list = categorical + numerical
    new_list.remove(c)
    modelC = train_model_with(dvC, df_train[new_list], y_train_binary)
    
    #accuracy on val dataset:
    accuracy1 = get_accuracy(dvC, modelC, df_val[new_list], y_val_binary)
    print("Model accuracy based on val data without",c,":", round(accuracy1,2))

# 29337 9779 9779

Model accuracy based on val data without: 0.79
Model accuracy based on val data without neighbourhood_group : 0.75
Model accuracy based on val data without room_type : 0.73
Model accuracy based on val data without latitude : 0.79
Model accuracy based on val data without longitude : 0.79
Model accuracy based on val data without minimum_nights : 0.79
Model accuracy based on val data without number_of_reviews : 0.79
Model accuracy based on val data without reviews_per_month : 0.79
Model accuracy based on val data without calculated_host_listings_count : 0.79
Model accuracy based on val data without availability_365 : 0.78


In [17]:
#Q6 Linear regression

def train_linear_regression_model(dv, dataFrame, y, alpha):
    dicts = dataFrame.to_dict(orient="records")
    X = dv.fit_transform(dicts)

    model = linear_model.Ridge(alpha=alpha)
    model.fit(X, y)
    return model

def get_prediction(dv1, model, df):
    dicts_val1 = df.to_dict(orient="records")
    X_val1 = dv1.fit_transform(dicts_val1)
    y_val_predict = model.predict(X_val1)
#     y_val_final_price_predicted = np.expm1(y_val_predict)
    return y_val_predict;

def rmse(y, y_pred):
    error = y-y_pred
    se = error ** 2
    mse = se.mean()
    return np.sqrt(mse)


for alpha in [0, 0.01, 0.1, 1, 10]:
    dv6 = DictVectorizer(sparse=False)
    linear_regression_model = train_linear_regression_model(dv6, df_train[categorical+numerical], y_train_log, alpha)
    y_val_log_Predicted = get_prediction(dv6, linear_regression_model, df_val[categorical+numerical])
    print("RMSE on linear regression model with alpha:",alpha,"is:", round(rmse(y_val_log, y_val_log_Predicted),3))


RMSE on linear regression model with alpha: 0 is: 0.497
RMSE on linear regression model with alpha: 0.01 is: 0.497
RMSE on linear regression model with alpha: 0.1 is: 0.497
RMSE on linear regression model with alpha: 1 is: 0.497
RMSE on linear regression model with alpha: 10 is: 0.498
